In [32]:
import pandas as pd
import numpy as np

### Задание 2.7
Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии. Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки.

Загрузите стандартный датасет об алмазах из библиотеки Seaborn:

In [33]:
import seaborn as sns
df = sns.load_dataset('diamonds')

Удалите часть признаков:

In [34]:
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

Закодируйте категориальные признаки:

In [35]:
df = pd.get_dummies(df, drop_first=True)

Логарифмируйте признаки:

In [36]:
df['carat'] = np.log(1+df['carat'])
df['price'] = np.log(1+df['price'])

Определите целевую переменную и предикторы:

In [37]:
X = df.drop(columns="price")
y = df["price"]

Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42.

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor). Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:

```
"loss": ["squared_error", "epsilon_insensitive"],
"penalty": ["elasticnet"],
"alpha": np.logspace(-3, 3, 10),
"l1_ratio": np.linspace(0, 1, 10),
"learning_rate": ["constant"],
"eta0": np.logspace(-4, -1, 4)
```

In [38]:
# библиотеки
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor

from sklearn import metrics #метрики

In [39]:
# разделение выборки
#разбиваем исходную выборку на тренировочную и валидационную в соотношении 0.33
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [44]:
param_grid = [{
        "loss": ["squared_error", "epsilon_insensitive"],
        "penalty": ["elasticnet"],
        "alpha": np.logspace(-3, 3, 10),
        "l1_ratio": np.linspace(0, 1, 10),
        "learning_rate": ["constant"],
        "eta0": np.logspace(-4, -1, 4)
        }]
grid_search_lg = GridSearchCV(
    estimator=SGDRegressor(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
) 

grid_search_lg.fit(X_train, y_train) 
print("accuracy на валидационном наборе: {:.2f}".format(grid_search_lg.score(X_valid, y_valid)))
y_valid_pred = grid_search_lg.predict(X_valid)
acc_le=metrics.mean_squared_error(y_valid, y_valid_pred)
print('MSE на валидационном наборе: {:.3f}'.format(acc_le))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lg.best_params_))

accuracy на валидационном наборе: 0.96
MSE на валидационном наборе: 0.044
Наилучшие значения гиперпараметров: {'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}
